# Notebook 06: Run Many Episodes (Batch Runner)

This notebook generates the **Safety-Transfer Hospital Benchmark v0.1 Dataset**.

**Protocol:**
1. Generate `NUM_WORLDS` unique hospital layouts using fixed seeds.
2. For each world, run `NUM_EPISODES` navigation tasks.
3. Save all logs and configuration to `data/dataset_v0.1`.

**Output Structure:**
```
data/dataset_v0.1/
  world_000/
  world_001/
    objects.json
    map.pgm
    episodes/
      episode_00_log.csv
      episode_01_log.csv
      ...
```

In [ ]:
import sys
import os
import json
import random
import numpy as np

# Add src to path
sys.path.append(os.path.abspath('../src'))

from world_gen.hospital_generator import HospitalGenerator
from simulation.episode_runner import EpisodeRunner

# Configuration
DATASET_DIR = "../data/dataset_v0.1"
NUM_WORLDS = 10  # Start small for demo, target 50+
EPISODES_PER_WORLD = 5
WIDTH_M = 20
HEIGHT_M = 20
RESOLUTION = 0.05

if not os.path.exists(DATASET_DIR):
    os.makedirs(DATASET_DIR)

## 1. Batch Execution Loop

In [ ]:
for w_idx in range(NUM_WORLDS):
    # 1. Generate World
    # Set seed for reproducibility
    seed = 42 + w_idx
    random.seed(seed)
    np.random.seed(seed)
    
    gen = HospitalGenerator(width=WIDTH_M, height=HEIGHT_M, resolution=RESOLUTION)
    gen.initialize_map()
    gen.generate_layout(num_wards=random.randint(2, 4))
    
    # Save World
    world_dir = os.path.join(DATASET_DIR, f"world_{w_idx:03d}")
    if not os.path.exists(world_dir):
        os.makedirs(world_dir)
    
    gen.save_to_json(os.path.join(world_dir, "objects.json"))
    gen.save_map_pgm(os.path.join(world_dir, "map"))
    
    # 2. Run Episodes
    episodes_dir = os.path.join(world_dir, "episodes")
    if not os.path.exists(episodes_dir):
        os.makedirs(episodes_dir)
        
    world_config = gen.to_dict()
    runner = EpisodeRunner(world_config, mode="mock")
    
    print(f"Generating World {w_idx:03d}...")
    
    for e_idx in range(EPISODES_PER_WORLD):
        # Random Start/Goal in free space (simplified)
        # For mock, we just pick corridor ends roughly
        start = (2.0, 10.0 + random.uniform(-1, 1), 0.0)
        goal = (18.0, 10.0 + random.uniform(-1, 1), 0.0)
        
        traj = runner.run_episode(start, goal, duration=40.0)
        
        # Save Log
        log_path = os.path.join(episodes_dir, f"episode_{e_idx:02d}_log.csv")
        runner.save_log(log_path)
        
        # Also save meta (start, goal) could go here
        
print("Dataset Generation Complete.")

## 2. Validation
Check that files were created.

In [ ]:
total_episodes = 0
for root, dirs, files in os.walk(DATASET_DIR):
    for file in files:
        if file.endswith("_log.csv"):
            total_episodes += 1
            
print(f"Total Episodes Generated: {total_episodes}")